The first section only have exploratory trials...

In [12]:
## This function will use CKAN api to get packages list plus the url for the resources under each entity 
## The results are stored in a pandas DF

import pprint
import requests
import pandas as pd

def getPackageList(url_api):
    r = requests.get(url_api)
    j = r.json()
    return(pd.DataFrame({"package_id": j['result']}))

def getResourceUrl(package_id):
    url = "https://opendata.porto.hackacity.eu" + "/api/3/action/package_show?id=" + package_id;  
    r = requests.get(url)
    j = r.json()
    return(j['result']['resources'][0]['url'])

In [ ]:
## Usage

url_api = "https://opendata.porto.hackacity.eu/api/3/action/package_list"
df = getPackageList(url_api)

df["resource_url"] = list(map(getResourceUrl, df["package_id"]))

#print(df)

In [4]:
## Get packageIds as a list

def getPackageIds(url):
    r = requests.get(url)
    j = r.json()
    return(j['result'])

In [ ]:
## Usage

url = "https://opendata.porto.hackacity.eu/api/3/action/package_list";
packageIdsList = getPackageIds(url)
#print(packageIdsList)

In [279]:
## For a list of packagesIds extract the available metadata and store the results in a pandas DF

from pandas.io.json import json_normalize

def normalizeJSON(url, package_id):
    url = url + package_id;  
    r = requests.get(url)
    j = r.json()
    return(json_normalize(j['result']))

def appendRowToDF(df1, df2):
    df = pd.DataFrame()
    if df1.empty: 
        df = df1.copy()
    else:
        df = pd.concat([df1,df2])
    return(df)

In [ ]:
## Usage

packagesMetadata = pd.DataFrame()
url = "https://opendata.porto.hackacity.eu/api/3/action/package_show?id="
for packageId in packageIdsList:
    row = normalizeJSON(url, packageId)
    packagesMetadata = appendRowToDF(packagesMetadata, row)  

In [45]:
## Request live data and store the result in a panda DF

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def getLiveData(api_url, package_id = None, device_id = None):
    if (package_id is None) and (device_id is not None):
        url = api_url + "/v2/entities?id=" + device_id
    elif (package_id is not None) and (device_id is None):
        url = api_url + "/v2/entities?type=" + package_id
    else:
        raise Exception("package_id or device_id required or are simultaneous defined!")

    print(url)
    
    r = requests.get(url)
    j = r.json()
    #pprint.pprint(j)
    return(json_normalize(j))

In [256]:
## Example with error
#api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
#df = getLiveData(api_url)

In [46]:
## Example without device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
package_id = "AirQualityObserved"
df = getLiveData(api_url, package_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?type=AirQualityObserved


In [51]:
## Example with device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
device_id = "urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54"
df = getLiveData(api_url, device_id = device_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?id=urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54


In [14]:
## Example 
## require getHistoricalData function declared below!!!

#resource_list = ['urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067']

#api_url = "http://history-data.urbanplatform.portodigital.pt"

#df = pd.DataFrame()
#for resource in resource_list:
#    print(resource)
#    df1 = getHistoricalData(api_url, device_id = resource, no=500)
#    #print(df1.columns)
#    if df.empty:
#        df = df1.copy()
#    else:
#        df = pd.concat([df,df1])
#    #print(df.columns)   


From this point forward are the sequential steps to get the IOT data...


In [16]:
## List IOT entities
import requests
import json

def getListIOTEntities(api_url):
    url = api_url + "/v2/types"
    r = requests.get(url)
    j = r.json()
    entity_list = []
    for item in j:
        #print(item['type'])
        entity_list.append(item['type'])
    return(entity_list)

In [17]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
entity_list = getListIOTEntities(api_url)
print(entity_list)

['AirQualityObserved', 'ArrivalEstimation', 'Device', 'Event', 'GtfsTransitFeedFile', 'NoiseLevelObserved', 'OffStreetParking', 'PointOfInterest', 'TrafficFlowObserved', 'Vehicle', 'WeatherForecast', 'WeatherObserved', 'gtfsAgency', 'gtfsCalendarDateRule', 'gtfsCalendarRule', 'gtfsRoute', 'gtfsService', 'gtfsStop']


In [18]:
## Reduce entity_list to the ones of interest (MANUAL STEP)

#1 entity_list = ['AirQualityObserved']
#2 entity_list = ['NoiseLevelObserved']
#3 entity_list = ['TrafficFlowObserved']
entity_list = ['WeatherObserved']

In [19]:
## Get list of all available resources/devices for an entity

import requests
import pprint

def getResourceList(api_url, entity_name):
    # First let's the ID's from the FiWare
    url = api_url + "/v2/entities?type=" + entity_name
    #print(url)
    r = requests.get(url, verify=False)
    j = r.json()
    #print(j)
    resources_list = []
    for resource in j: 
      resources_list.append(resource['id'])
    return(resources_list)

In [20]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"

resource_list = []
for entity in entity_list:
    print(entity)
    resource_list.append(getResourceList(api_url, entity))    

WeatherObserved


/home/cha1av/dev/hackacity/env/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#print(resource_list)

In [21]:
#Flat resource_list
resource_list_flat = [item for sublist in resource_list for item in sublist]

In [23]:
#Remove stuff that is giving errors
#resource_list_flat.remove('Trindade_1_Rotacao')

In [22]:
print(resource_list_flat)

['urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTUGA71', 'urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTOPO9', 'urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTOPO4', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5ad615bb3da2520542bc87fb', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5ad616a53da2520542bc8859', 'urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTO36', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b87a6eb0dde591949bb8b5e', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b87a6b60dde591949bb8b5b', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b87a6cc0dde591949bb8b5c', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b87a6dc0dde591949bb8b5d', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b97a1bde521e3053085c08c', 'urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b72dbfa06599b05e9a74f2

In [23]:
## Request historical data with resource_id and store the results as a panda DF for a single device

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def convertJSONHistoricalToDF(j, entity = None):
    print(entity)
    if (entity is (None or "NoiseLevelObserved" or "AirQualityObserved")):
        df = pd.DataFrame()
        for i in range(0,len(j['data']['attributes'])):
            df[j['data']['attributes'][i]['attrName']] = j['data']['attributes'][i]['values']
        ## expand location attributes
        #for key in df['location'][0].keys():
        #    df[key] = str(df['location'][0][key])
        df['long'] = df['location'][0]['coordinates'][0]
        df['lat'] = df['location'][0]['coordinates'][1]
        df['device_id'] = j['data']['entityId']
        #print(df)
    elif (entity is "TrafficFlowObserved"):
        df = pd.DataFrame()
        #TODO
    elif (entity is "WeatherObserved"):
        df = pd.DataFrame()
        for i in range(0, len(j['data']['attributes'])):
            df[j['data']['attributes'][i]['attrName']] = j['data']['attributes'][i]['values']
        df['long'] = df['location'][0]['coordinates'][0]
        df['lat'] = df['location'][0]['coordinates'][1]
        df['device_id'] = j['data']['entityId']
    else:
        raise Exception("Wrong json template specified")
    return(df)

def getHistoricalData(api_url, device_id=None, no=20, output="json"):
    if (device_id is not None):
        url = api_url + "/v2/entities/" + device_id + "?limit=" + str(no)
    else:
        raise Exception("device_id is required")
    r = requests.get(url)
    j = r.json()

    if "error" not in j:        
        if (output is "json"):
            return (j)
        elif (output is "df"):
            return (convertJSONHistoricalToDF(j, entity))
        else:
            raise Exception("invalid output")
            
#def getHistoricalData(api_url, device_id = None, no = 20):
#    if (device_id is not None):
#        url = api_url + "/v2/entities/" + device_id + "?limit=" + str(no)
#    else:
#        raise Exception("device_id is required")
#
#    #print(url)
#    r = requests.get(url)
#    j = r.json()
#    #json_normalize(j['data'], record_path=['attributes']).transpose()
#    return(convertJSONHistoricalToDF(j, entity))

In [30]:
import pandas as pd 

api_url = "http://history-data.urbanplatform.portodigital.pt"

df = pd.DataFrame()
for resource in resource_list_flat:
    print(entity)
    print(resource)
    df1 = getHistoricalData(api_url, device_id = resource, no=500, output="df")
    print(df1)
    #print(df1.columns)
    if df1 is not None:
        if df.empty:
            df = df1.copy()
        else:
            df = pd.concat([df,df1], sort=False)
        #print(df.columns)  

WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTUGA71
None
WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTOPO9
None
WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:communityStations:IPORTOPO4
None
WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a
WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1004.4         None   
1    None  None  None              1017.4         None   
2    None  None  None              1017.3         None   
3    None  None  None              1017.4         None   
4    None  None  None              1017.5         None   
5    None  None  None              1017.5         None   
6    None  None  None              1017.5         None   
7    None  None  None              1017.5         None   
8    None  None  None              1017.5         None   
9    None  None  None              101

WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None               733.4         None   
1    None  None  None               732.4         None   
2    None  None  None               730.5         None   
3    None  None  None               732.5         None   
4    None  None  None               734.6         None   
5    None  None  None               734.8         None   
6    None  None  None               735.1         None   
7    None  None  None               734.8         None   
8    None  None  None               735.4         None   
9    None  None  None               735.3         None   
10   None  None  None               733.0         None   
11   None  None  None               734.3         None   
12   None  None  None               734.4         None   
13   None  None  None               734.4         None   
14   None  None  None               731.0         None   
15   None  None  None               731.0         None  

WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1022.6         None   
1    None  None  None              1022.7         None   
2    None  None  None              1022.7         None   
3    None  None  None              1022.7         None   
4    None  None  None              1022.7         None   
5    None  None  None              1022.7         None   
6    None  None  None              1022.7         None   
7    None  None  None              1022.7         None   
8    None  None  None              1022.7         None   
9    None  None  None              1022.6         None   
10   None  None  None              1022.6         None   
11   None  None  None              1022.6         None   
12   None  None  None              1022.6         None   
13   None  None  None              1022.6         None   
14   None  None  None              1022.6         None   
15   None  None  None              1022.6         None  

None
WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b87a6eb0dde591949bb8b5e
WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1023.3         None   
1    None  None  None              1023.4         None   
2    None  None  None              1023.4         None   
3    None  None  None              1023.4         None   
4    None  None  None              1023.4         None   
5    None  None  None              1023.4         None   
6    None  None  None              1023.4         None   
7    None  None  None              1023.4         None   
8    None  None  None              1023.4         None   
9    None  None  None              1023.4         None   
10   None  None  None              1023.3         None   
11   None  None  None              1023.3         None   
12   None  None  None              1023.3         None   
13   None  None  None              1023.3         None   
14   None  None  

WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1022.9         None   
1    None  None  None              1022.9         None   
2    None  None  None              1022.9         None   
3    None  None  None              1022.9         None   
4    None  None  None              1022.9         None   
5    None  None  None              1022.9         None   
6    None  None  None              1022.9         None   
7    None  None  None              1022.8         None   
8    None  None  None              1022.8         None   
9    None  None  None              1022.8         None   
10   None  None  None              1022.7         None   
11   None  None  None              1022.7         None   
12   None  None  None              1022.7         None   
13   None  None  None              1022.6         None   
14   None  None  None              1022.6         None   
15   None  None  None              1022.6         None  

WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1023.2         None   
1    None  None  None              1023.3         None   
2    None  None  None              1023.3         None   
3    None  None  None              1023.3         None   
4    None  None  None              1023.3         None   
5    None  None  None              1023.3         None   
6    None  None  None              1023.3         None   
7    None  None  None              1023.3         None   
8    None  None  None              1023.3         None   
9    None  None  None              1023.2         None   
10   None  None  None              1023.2         None   
11   None  None  None              1023.2         None   
12   None  None  None              1023.3         None   
13   None  None  None              1023.2         None   
14   None  None  None              1023.2         None   
15   None  None  None              1023.2         None  

WeatherObserved
      PM1  PM10  PM25  barometricPressure dataProvider  \
0    None  None  None              1023.4         None   
1    None  None  None              1023.5         None   
2    None  None  None              1023.5         None   
3    None  None  None              1023.5         None   
4    None  None  None              1023.5         None   
5    None  None  None              1023.5         None   
6    None  None  None              1023.5         None   
7    None  None  None              1023.5         None   
8    None  None  None              1023.5         None   
9    None  None  None              1023.5         None   
10   None  None  None              1023.5         None   
11   None  None  None              1023.5         None   
12   None  None  None              1023.4         None   
13   None  None  None              1023.5         None   
14   None  None  None              1023.4         None   
15   None  None  None              1023.4         None  

WeatherObserved
    PM1  PM10  PM25 barometricPressure dataProvider             dateObserved  \
0   NaN   NaN   NaN               None         None  2019-02-18T01:52:52.000   
1   0.0   0.0   0.0               None         None  2019-03-20T04:23:01.000   
2   0.0   0.0   0.0               None         None  2019-03-20T04:38:26.000   
3   0.0   0.0   0.0               None         None  2019-03-20T04:44:27.000   
4   0.0   0.0   0.0               None         None  2019-03-20T04:54:30.000   
5   0.0   0.0   0.0               None         None  2019-03-20T04:57:35.000   
6   0.0   0.0   0.0               None         None  2019-03-20T05:07:07.000   
7   0.0   0.0   0.0               None         None  2019-03-20T05:13:17.000   
8   0.0   0.0   0.0               None         None  2019-03-20T05:23:24.000   
9   0.0   0.0   0.0               None         None  2019-03-20T05:26:44.000   
10  0.0   0.0   0.0               None         None  2019-03-20T05:36:47.000   
11  0.0   0.0   0.0     

WeatherObserved
    PM1  PM10  PM25 barometricPressure dataProvider             dateObserved  \
0   NaN   NaN   NaN               None         None  2019-03-05T23:08:44.000   
1   NaN   NaN   NaN               None         None  2019-03-06T05:27:03.000   
2   0.0   0.0   0.0               None         None  2019-03-20T04:29:10.000   
3   0.0   0.0   0.0               None         None  2019-03-20T04:38:19.000   
4   0.0   0.0   0.0               None         None  2019-03-20T04:54:25.000   
5   0.0   0.0   0.0               None         None  2019-03-20T04:57:31.000   
6   0.0   0.0   0.0               None         None  2019-03-20T05:07:02.000   
7   0.0   0.0   0.0               None         None  2019-03-20T05:13:12.000   
8   0.0   0.0   0.0               None         None  2019-03-20T05:26:58.000   
9   0.0   0.0   0.0               None         None  2019-03-20T05:37:02.000   
10  0.0   0.0   0.0               None         None  2019-03-20T05:43:18.000   
11  0.0   0.0   0.0     

None
WeatherObserved
urn:ngsi-ld:WeatherObserved:porto:weather:ubiwhere:5b97a1a6e521e3053085c067
WeatherObserved
    PM1  PM10  PM25 barometricPressure dataProvider             dateObserved  \
0     0     0     0               None         None  2019-03-20T04:29:12.000   
1     0     0     0               None         None  2019-03-20T04:38:21.000   
2     0     0     0               None         None  2019-03-20T05:13:15.000   
3     0     0     0               None         None  2019-03-20T05:26:53.000   
4     0     0     0               None         None  2019-03-20T05:36:57.000   
5     0     0     0               None         None  2019-03-20T05:43:08.000   
6     0     0     0               None         None  2019-03-20T05:52:59.000   
7     0     0     0               None         None  2019-03-20T06:24:32.000   
8     0     0     0               None         None  2019-03-20T06:26:15.000   
9     0     0     0               None         None  2019-03-20T06:37:27.000   
10    0

In [31]:
# DF operations

import  time
import datetime

# Convert timestamp to unixtime
def convert_df_to_unix(s):
    time_mask = "%Y-%m-%dT%H:%M:%S.%f"
    return(time.mktime(datetime.datetime.strptime(s, time_mask).timetuple()))
 


In [32]:
df['time'] = list(map(convert_df_to_unix, df['dateObserved']))

In [38]:
## Save columns to csv in a custom order
mandatory_column_list = ["time", "lat", "long"]

if (entity is "AirQualityObserved"):
    data_column_list = ["CO", "NO2", "O3", "Ox", "PM1", "PM10", "PM25"]

    for data_column in data_column_list:
        filepath = "./output/" + "IOT_" + "AirQuality_" + data_column + ".csv"
        print(filepath)
        df.to_csv(filepath, columns = mandatory_column_list + [data_column], 
                  index = False, na_rep = "", header = True)

elif (entity is "NoiseLevelObserved"):
    data_column_list = ["LAeq"]
    
    for data_column in data_column_list:
        filepath = "./output/" + "IOT_" + "NoiseLevelObserved_" + data_column + ".csv"
        print(filepath)
        df.to_csv(filepath, columns = mandatory_column_list + [data_column], 
                  index = False, na_rep = "", header = True)

elif (entity is "TrafficFlowObserved"):
    data_column_list = ["LAeq"]
    
    for data_column in data_column_list:
        filepath = "./output/" + "IOT_" + "TrafficFlowObserved_" + data_column + ".csv"
        print(filepath)
        df.to_csv(filepath, columns = mandatory_column_list + [data_column], 
                  index = False, na_rep = "", header = True)

elif (entity is "WeatherObserved"):
    data_column_list = ['barometricPressure', 'precipitation', 'relativeHumidity',
       'solarRadiation', 'temperature', 'windDirection', 'windSpeed']
    
    for data_column in data_column_list:
        filepath = "./output/" + "IOT_" + "WeatherObserved_" + data_column + ".csv"
        print(filepath)
        df.to_csv(filepath, columns = mandatory_column_list + [data_column], 
                  index = False, na_rep = "", header = True)
        

./output/IOT_WeatherObserved_barometricPressure.csv
./output/IOT_WeatherObserved_precipitation.csv
./output/IOT_WeatherObserved_relativeHumidity.csv
./output/IOT_WeatherObserved_solarRadiation.csv
./output/IOT_WeatherObserved_temperature.csv
./output/IOT_WeatherObserved_windDirection.csv
./output/IOT_WeatherObserved_windSpeed.csv


In [36]:
print(df.columns)

Index(['PM1', 'PM10', 'PM25', 'barometricPressure', 'dataProvider',
       'dateObserved', 'location', 'precipitation', 'relativeHumidity',
       'solarRadiation', 'source', 'temperature', 'windDirection', 'windSpeed',
       'long', 'lat', 'device_id', 'time'],
      dtype='object')
